# TEXT GENERATION USING GPT-2

In [ ]:
# Install the transformers package if you haven't already
# !pip install transformers

In [ ]:
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model_name = "gpt2-xl"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name).to(device)